In [37]:
# load the necessary packages 

import os
import scipy
import shutil
import numpy as np
import pandas as pd
import xarray as xr
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
cartopy.config['data_dir'] = os.getenv('CARTOPY_DIR', cartopy.config.get('data_dir'))

import plotly.express as px
import geopandas as gpd
from scipy.spatial import cKDTree

import imageio
import os
import io
import requests

import datetime as dt
from dateutil.relativedelta import *

import netCDF4
from netCDF4 import Dataset, num2date

import cmocean

import seaborn as sns
import pymannkendall as mk

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from matplotlib import pyplot as plt
%matplotlib inline

from PIL import Image  # To create gifs
from shapely.geometry import Point
from shapely.geometry import LineString

In [39]:
# Loding the cvs with all the stats for all positions 
all_stats = pd.read_csv('combined_stats_during.csv')
all_stats

,var,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9
0,temp_pos4_Mean,30.452438,30.304798,29.783731,29.542430,29.346487,29.207323,29.068980,29.126822,29.378609
1,temp_pos4_Max,30.649218,30.611778,30.001358,29.683575,29.465958,29.295725,29.155552,29.307812,29.504040
2,temp_pos4_Min,30.110304,29.974293,29.457851,29.333520,29.189405,29.037859,28.946226,28.961296,29.231780
3,temp_pos5_Mean,30.368689,30.254457,29.811983,29.473957,29.264202,29.130770,29.010042,29.010029,29.255510
4,temp_pos5_Max,30.649218,30.611778,30.013517,29.621355,29.465958,29.275011,29.162775,29.289953,29.492980
...,...,...,...,...,...,...,...,...,...,...
202,Chl_pos35_Max,0.068210,0.058190,0.057333,0.053044,0.042135,0.043165,0.039545,0.036130,0.039129
203,Chl_pos35_Min,0.027515,0.025786,0.022868,0.021744,0.017938,0.015646,0.013920,0.012721,0.019063
204,Chl_pos36_Mean,0.028977,0.029192,0.028755,0.030848,0.028296,0.028486,0.028939,0.031232,0.034393
205,Chl_pos36_Max,0.032690,0.038512,0.042875,0.044485,0.034882,0.036477,0.034582,0.036198,0.039238


In [41]:
# Splitting the first column
parse = all_stats['var'].iloc[0]
parse_split = parse.split("_")
parse_split[0]

'temp'

In [43]:
var_name = []
position = []
stats = []
for var in range(len(all_stats)):
    parse = all_stats['var'].iloc[var]
    parse_split = parse.split("_")
    parse_split
    var_name.append(parse_split[0])
    position.append(parse_split[1])
    stats.append(parse_split[2])

In [45]:
all_stats['var_name']  = var_name
all_stats['position']  = position
all_stats['stats']  = stats
all_stats

,var,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,var_name,position,stats
0,temp_pos4_Mean,30.452438,30.304798,29.783731,29.542430,29.346487,29.207323,29.068980,29.126822,29.378609,temp,pos4,Mean
1,temp_pos4_Max,30.649218,30.611778,30.001358,29.683575,29.465958,29.295725,29.155552,29.307812,29.504040,temp,pos4,Max
2,temp_pos4_Min,30.110304,29.974293,29.457851,29.333520,29.189405,29.037859,28.946226,28.961296,29.231780,temp,pos4,Min
3,temp_pos5_Mean,30.368689,30.254457,29.811983,29.473957,29.264202,29.130770,29.010042,29.010029,29.255510,temp,pos5,Mean
4,temp_pos5_Max,30.649218,30.611778,30.013517,29.621355,29.465958,29.275011,29.162775,29.289953,29.492980,temp,pos5,Max
...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Chl_pos35_Max,0.068210,0.058190,0.057333,0.053044,0.042135,0.043165,0.039545,0.036130,0.039129,Chl,pos35,Max
203,Chl_pos35_Min,0.027515,0.025786,0.022868,0.021744,0.017938,0.015646,0.013920,0.012721,0.019063,Chl,pos35,Min
204,Chl_pos36_Mean,0.028977,0.029192,0.028755,0.030848,0.028296,0.028486,0.028939,0.031232,0.034393,Chl,pos36,Mean
205,Chl_pos36_Max,0.032690,0.038512,0.042875,0.044485,0.034882,0.036477,0.034582,0.036198,0.039238,Chl,pos36,Max


In [85]:
def get_indices(lst, targets):
    return list(filter(lambda x: lst[x] in targets, range(len(lst))))
 
shift = np.zeros(len(all_stats),dtype=int)
target = ['pos7', 'pos8', 'pos9', 'pos10']
idx = get_indices(position, target)
shift[idx] = -1

target = ['pos11', 'pos12', 'pos13', 'pos14']
idx = get_indices(position, target)
shift[idx] = -2

target = ['pos15', 'pos16', 'pos17', 'pos18']
idx = get_indices(position, target)
shift[idx] = -3


target = ['pos19', 'pos20', 'pos21', 'pos22']
idx = get_indices(position, target)
shift[idx] = -4

target = ['pos23', 'pos24']
idx = get_indices(position, target)
shift[idx] = -5

target = ['pos35', 'pos36']
idx = get_indices(position, target)
shift[idx] = -8

In [87]:
all_stats['shift']  = shift

,var,Day 1,Day 2,Day 3,Day 4,Day 5,Day 6,Day 7,Day 8,Day 9,var_name,position,stats,shift
0,temp_pos4_Mean,30.452438,30.304798,29.783731,29.542430,29.346487,29.207323,29.068980,29.126822,29.378609,temp,pos4,Mean,0
1,temp_pos4_Max,30.649218,30.611778,30.001358,29.683575,29.465958,29.295725,29.155552,29.307812,29.504040,temp,pos4,Max,0
2,temp_pos4_Min,30.110304,29.974293,29.457851,29.333520,29.189405,29.037859,28.946226,28.961296,29.231780,temp,pos4,Min,0
3,temp_pos5_Mean,30.368689,30.254457,29.811983,29.473957,29.264202,29.130770,29.010042,29.010029,29.255510,temp,pos5,Mean,0
4,temp_pos5_Max,30.649218,30.611778,30.013517,29.621355,29.465958,29.275011,29.162775,29.289953,29.492980,temp,pos5,Max,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202,Chl_pos35_Max,0.068210,0.058190,0.057333,0.053044,0.042135,0.043165,0.039545,0.036130,0.039129,Chl,pos35,Max,-8
203,Chl_pos35_Min,0.027515,0.025786,0.022868,0.021744,0.017938,0.015646,0.013920,0.012721,0.019063,Chl,pos35,Min,-8
204,Chl_pos36_Mean,0.028977,0.029192,0.028755,0.030848,0.028296,0.028486,0.028939,0.031232,0.034393,Chl,pos36,Mean,-8
205,Chl_pos36_Max,0.032690,0.038512,0.042875,0.044485,0.034882,0.036477,0.034582,0.036198,0.039238,Chl,pos36,Max,-8


In [91]:
all_stats.to_csv('allstats.csv')